## Logistic Matrix Factorization

Maximize the following :

$$
\mathcal{L}(W, H) = \sum_{i=1}^I \sum_{j=1}^J M(i,j) ( Y(i,j) \log (\sigma(W(i) H(j))) + (1 - Y(i,j)) \log(1 - \sigma(W(i) H(j))) )
$$

Observed $I\times J$ binary matrix with possibly missing entries
$Y(i,j) \in \{0,1\}$

Mask Matrix
$M(i,j) = 1$ if $Y(i,j)$ is observed, $M(i,j) = 0$ if $Y(i,j)$ is not observed


Here:

$\sigma(x)$ is the sigmoid function defined as
\begin{eqnarray}
\sigma(x) & = & \frac{1}{1+e^{-x}}
\end{eqnarray}


### Properties of the sigmoid function
Note that

\begin{eqnarray}
\sigma(x) & = & \frac{e^x}{(1+e^{-x})e^x} = \frac{e^x}{1+e^{x}} \\
1 - \sigma(x) & = & 1 - \frac{e^x}{1+e^{x}} = \frac{1+e^{x} - e^x}{1+e^{x}} = \frac{1}{1+e^{x}}
\end{eqnarray}

\begin{eqnarray}
\sigma'(x) & = & \frac{e^x(1+e^{x}) - e^{x} e^x}{(1+e^{x})^2} = \frac{e^x}{1+e^{x}}\frac{1}{1+e^{x}} = \sigma(x) (1-\sigma(x))
\end{eqnarray}

\begin{eqnarray}
\log \sigma(x) & = & -\log(1+e^{-x}) = x - \log(1+e^{x}) \\
\log(1 - \sigma(x)) & = &  -\log({1+e^{x}})
\end{eqnarray}



In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt


# Generate a random logistic regression problem

def sigmoid(t):
    return np.exp(t)/(1+np.exp(t))

I = 5
J = 10

# Random Mask 
M = np.random.rand(I,J)<0.8

# Random Parameters
W = np.random.randn(I,1)
H = np.random.randn(1,J)

Y = np.zeros((I,J))
# Generate class labels
pi = sigmoid(W*H)

for i in range(I):
    for j in range(J):
        if not M[i,j]:
            Y[i,j] = np.nan
        else:
            Y[i,j] = 1 if pi[i,j] < np.random.rand() else 0
            

#Solution starts here

print('W and H used to generate problem matrix: W=' , W, ', H =', H)

#Assign new randoms to start gradient
W = np.random.randn(I,1)
H = np.random.randn(1,J)

def deriveH(i,j):
    if not M[i,j]:
        return 0
    else:
        (W[i]*(((Y[i,j]-1)*np.exp(W[i]*H[0][j]))+Y[i,j]))/(np.exp(H[0][j]*W[i]) + 1)

def deriveW(i,j):
    if not M[i,j]:
        return 0
    else:
        (H[0][j]*(((Y[i,j]-1)*np.exp(W[i]*H[0][j]))+Y[i,j]))/(np.exp(H[0][j]*W[i]) + 1)
        



        
trials = 50
epoch = 5000
eta = 0.00003366
for t in range(trials):
    for e in range(epoch):
        for i in range(I):
            for j in range(J):
                derW = deriveW(i,j)
                if(derW is not None):
                    W[i] = W[i] - (eta*derW)
                derH = deriveH(i,j)
                if(derH is not None):
                    H[0][j] = H[0][j] - (eta*derH)
                
                
                
print('W and H found after gradient solution: W=' , W, ', H =', H)

Task: 
Given $Y$ and $M$ only find a good $W$ and $H$ by maximizing the objective $\mathcal{L}$